### Census Trade Data

September 30, 2020

Brian Dew

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

from io import BytesIO, TextIOWrapper
from zipfile import ZipFile

### Goods Trade by Partner

In [2]:
# Retrieve trade data
url = ('https://www.census.gov/foreign-trade/balance/country.xlsx')
df = pd.read_excel(url)

In [3]:
# Melt the DataFrame to have one row per country per year per month
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 
          'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
import_columns = ['I' + month for month in months]
export_columns = ['E' + month for month in months]

# Melt imports
dfi = df.melt(id_vars=['year', 'CTY_CODE', 'CTYNAME'], value_vars=import_columns,
                     var_name='Month', value_name='Imports')
dfi['date'] = pd.to_datetime(dfi.Month.str[1:] + ' ' + dfi.year.astype('str'))

# Melt exports
dfe = df.melt(id_vars=['year', 'CTY_CODE', 'CTYNAME'], value_vars=export_columns,
                     var_name='Month', value_name='Exports')
dfe['date'] = pd.to_datetime(dfe.Month.str[1:] + ' ' + dfe.year.astype('str'))

# Combine imports and exports
dfc = pd.merge(dfi, dfe, on=['CTY_CODE', 'CTYNAME', 'date'])

res = (dfc[['date', 'CTY_CODE', 'CTYNAME', 'Imports', 'Exports']]
       .set_index(['CTYNAME', 'date']).sort_index())
res.replace(0, np.nan, inplace=True)
res.to_csv(data_dir / 'trade_partner.csv')

In [4]:
# Trade balance by partner chart
res = pd.read_csv(data_dir / 'trade_partner.csv', parse_dates=True, 
                  index_col=['CTYNAME', 'date'])
res['Balance'] = res['Exports'] - res['Imports']
res['Total'] = res['Exports'] + res['Imports']
s = ['World, Not Seasonally Adjusted', 'Canada', 'Mexico', 
     'European Union', 'China', 'Asia']
data = res.query('CTYNAME in @s')['Balance'].unstack().T.loc['1989':].dropna(how='all')
tot = data['World, Not Seasonally Adjusted']
data['Asia ex. China'] = data['Asia'] - data['China']
data = data[['Canada', 'Mexico', 'China', 'Asia ex. China', 'European Union']]

gdp = pd.read_csv(data_dir / 'gdp_monthly.csv', parse_dates=True, 
                  index_col='date').rolling(12).mean()['A191RC']
bal = (data.rolling(12).sum()
           .divide(gdp, axis=0)
           .dropna(how='all') * 100)
bal.columns.name = ''
bal.to_csv(data_dir / 'trade_bal_partner.csv', index_label='date')

# End nodes
colors = {'Canada' : 'green!90!blue', 'Mexico' : 'violet', 
          'China' : 'darkgray', 'Asia ex. China' : 'cyan!80!white', 
          'European Union' : 'blue!70!black'}

adj = node_adj(bal)
smax = bal.iloc[-1].idxmax()
adj[smax] += 0.35
date = {n: None for n in bal.columns}
date[smax] = 'm'
nodes = '\n'.join([end_node(bal[i], colors[i], 
                            date=date[i], offset=adj[i]) 
                   for i in colors.keys()])
write_txt(text_dir / 'trade_bal_partner_nodes.txt', nodes)

In [5]:
# Trade in goods, small multiples
res = pd.read_csv(data_dir / 'trade_partner.csv', parse_dates=True, 
                  index_col=['date', 'CTYNAME'])
res['Balance'] = res['Exports'] - res['Imports']
slist = ['Imports', 'Exports', 'Balance']
clist = ['Canada', 'Mexico', 'Japan', 'European Union', 'China', 
         'South and Central America']
df = res.query('CTYNAME in @clist')[slist].rolling(12).sum().unstack()
df.columns = [' '.join(col).strip().replace('European Union', 'EU')
                 .replace('South and Central America', 'SA') 
              for col in df.columns.values]
dft = (df.divide(gdp, axis=0) * 100).dropna(how='all')
dft.to_csv(data_dir / 'trade_partner_goods.csv', index_label='date')

In [6]:
# Imports and Exports by Partner Text
res = pd.read_csv(data_dir / 'trade_partner.csv', parse_dates=True, 
                  index_col=['CTYNAME', 'date'])
res['Balance'] = res['Exports'] - res['Imports']
# Imports Text
totch = (res.loc['World, Not Seasonally Adjusted', 'Imports']
         .rolling(12).sum().divide(gdp, axis=0) * 100).dropna()

totch89 = value_text(totch.loc['1989-12-01'], 'plain')
totchlt = value_text(totch.iloc[-1], 'plain')
ltdt = dtxt(totch.index[-1])['mon1']

cn_imp = (res.loc['China', 'Imports']
         .rolling(12).sum().divide(gdp, axis=0) * 100).dropna()
maxdate = cn_imp.idxmax()
maxdt = maxdate.year
cn_ch = value_text(cn_imp[maxdate] - cn_imp.loc['1989-12-01'], 'plain')
cnlt = value_text(cn_imp.iloc[-1], 'plain')
cn_lt = value_text(cn_imp[maxdate] - cn_imp.iloc[-1], 'plain', ptype='pp')
mx_imp = (res.loc['Mexico', 'Imports']
         .rolling(12).sum().divide(gdp, axis=0) * 100).dropna()
mx_ch = value_text(mx_imp.iloc[-1] - mx_imp.loc['1989-12-01'], 'increase_by')
jp_imp = (res.loc['Japan', 'Imports']
         .rolling(12).sum().divide(gdp, axis=0) * 100).dropna()
jp_ch = (value_text(jp_imp.iloc[-1] - jp_imp.loc['1989-12-01'], 'increase_by', 
                    casual=True).replace('fell', 'fallen'))

# Exports text
totex = res.loc['World, Not Seasonally Adjusted', 'Exports'].rolling(12).sum()
exgdp = (totex.divide(gdp, axis=0) * 100).dropna()
totchex = value_text(exgdp.iloc[-1] - exgdp.loc['1989-12-01'], 'increase_by')
ctylist = ['Canada', 'China', 'Mexico']
ctydf = (res.query('CTYNAME in @ctylist')['Exports']
            .unstack().T.rolling(12).sum().dropna())
octy = ([cty for cty in (ctydf.iloc[-1].sort_values(ascending=False)).index])

top3 = ctydf.iloc[-1].sum()
topsh = value_text((top3 / totex.dropna().iloc[-1]) * 100, 'plain')
top3gdp = value_text((top3 / gdp.loc[ctydf.index[-1]]) * 100, 'plain')
eu = res.loc['European Union', 'Exports'].rolling(12).sum()
eut = value_text((eu.dropna().iloc[-1] / 
                  gdp.loc[ctydf.index[-1]]) * 100, 'plain')

text = (f'Imports of goods have increased from {totch89} of GDP in '+
        f'1989 to {totchlt} of GDP in the year ending {ltdt}. '+
        f'Goods imports from China increased by {cn_ch} '+
        f'of GDP from 1989 to {maxdt}, and have since fallen by '+
        f'{cn_lt} to {cnlt} of GDP. Goods imports from Mexico have '+
        f'{mx_ch} of GDP since 1989. Goods imports '+
        f'from Japan have {jp_ch} of GDP.\n\n Exports '+
        f'of goods have {totchex} of GDP since 1989. '+
        f'The largest buyers of US-made goods are {octy[0]}, {octy[1]}, '+
        f'and {octy[2]}. Exports to these three countries make up '+
        f'{topsh} of exports and are equivalent to '+
        f'{top3gdp} of GDP over the year ending {ltdt}. Exports '+
        f'to the European Union currently total {eut} of GDP.')
write_txt(text_dir / 'trade_parter_imp_exp.txt', text)
print(text)

Imports of goods have increased from 8.4 percent of GDP in 1989 to 11.2 percent of GDP in the year ending April 2024. Goods imports from China increased by 2.5 percent of GDP from 1989 to 2015, and have since fallen by 1.2 percentage points to 1.5 percent of GDP. Goods imports from Mexico have increased by 1.3 percent of GDP since 1989. Goods imports from Japan have fallen by 1.1 percent of GDP.

 Exports of goods have increased by 0.8 percent of GDP since 1989. The largest buyers of US-made goods are Canada, Mexico, and China. Exports to these three countries make up 40.7 percent of exports and are equivalent to three percent of GDP over the year ending April 2024. Exports to the European Union currently total 1.3 percent of GDP.


In [7]:
bal = pd.read_csv(data_dir / 'trade_bal_partner.csv', parse_dates=True, 
                  index_col='date')
bal18 = value_text(abs(bal.loc['2018-12-01', 'China']), 'plain')
ballt = value_text(abs(bal['China'].iloc[-1]), 'plain')
totch = (res.loc['World, Not Seasonally Adjusted', 'Balance']
         .rolling(12).sum().divide(gdp, axis=0) * 100).dropna()
totlt = value_text(abs(totch.iloc[-1]), 'plain')
balch = value_text(abs(totch.iloc[-1]) - abs(totch.loc['1989-12-01']), 
                   'increase_by')

# Trade Balance Text 1
text = (f'Since 1989, the US goods trade deficit has {balch} of '+
        f'GDP, to {totlt} percent of GDP. '+
        f'In 2018, the deficit with China was {bal18} of GDP, '+
        f'but it has fallen to {ballt} of GDP. ')
write_txt(text_dir / 'trade_parter_bal1.txt', text)
print(text)

Since 1989, the US goods trade deficit has increased by two percent of GDP, to 3.9 percent percent of GDP. In 2018, the deficit with China was two percent of GDP, but it has fallen to one percent of GDP. 


In [8]:
eu = res.loc['European Union', 'Balance'].rolling(12).sum()
eugdp = ((eu / gdp) * 100).dropna()
eut = value_text(abs(eugdp.iloc[-1]), 'plain')
mx = res.loc['Mexico', 'Balance'].rolling(12).sum()
mxgdp = ((mx / gdp) * 100).dropna()
mxt = value_text(abs(mxgdp.iloc[-1]), 'plain')
sa = res.loc['South and Central America', 'Balance'].rolling(12).sum()
sagdp = ((sa / gdp) * 100).dropna()
sat = value_text(sagdp.iloc[-1], 'plain')

# Trade Balance Text 2
text = ('The US also runs a trade deficit with the European '+
        'Union. In 1997, trade between the EU and US was relatively balanced. '+
        f'In the latest data, the goods trade deficit with the EU is {eut} of GDP.\n\n '+
        f'The US trade deficit with Mexico is currently {mxt} of GDP. '+
        'In the early 1990s, the US had a trade surplus with Mexico. '+
        'The US has a surplus with south and central American '+
        f'countries, equivalent to {sat} of GDP. ')
write_txt(text_dir / 'trade_parter_bal2.txt', text)
print(text)

The US also runs a trade deficit with the European Union. In 1997, trade between the EU and US was relatively balanced. In the latest data, the goods trade deficit with the EU is 0.8 percent of GDP.

 The US trade deficit with Mexico is currently 0.6 percent of GDP. In the early 1990s, the US had a trade surplus with Mexico. The US has a surplus with south and central American countries, equivalent to 0.2 percent of GDP. 


### Trade by Partner from API

In [10]:
# Retrieve data from Census API
d = {}
for flow in ['imports', 'exports']:
    base = f'https://api.census.gov/data/timeseries/intltrade/{flow}/hs'
    params = ['CTY_CODE', 'CTY_NAME', 'ALL_VAL_MO']
    if flow == 'imports':
        params[2] = 'GEN_VAL_MO'
    param = ','.join(params)
    url = f'{base}?get={param}&key={census_key}&time=from+1992-01'
    d[flow] = requests.get(url).json()
    
i = d['imports']
df = pd.DataFrame(i[1:], columns=i[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
imp = (df.rename({'GEN_VAL_MO': 'Imports'}, axis=1)
         .set_index(['Name', 'Date']).Imports.astype(float)
         .drop_duplicates(keep='last'))

e = d['exports']
df = pd.DataFrame(e[1:], columns=e[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
exp = (df.rename({'ALL_VAL_MO': 'Exports'}, axis=1)
         .set_index(['Name', 'Date']).Exports.astype(float)
         .drop_duplicates(keep='last'))

data = ((pd.DataFrame([imp, exp]).T / 1000000)
           .rename({'Korea, South': 'South Korea',
                    'Total For All Countries': 'Total, All Countries'}))

data.sort_index().to_csv(data_dir / 'trade_master.csv')

In [11]:
data = pd.read_csv(data_dir / 'trade_master.csv', 
                   parse_dates=True, index_col=['Name', 'Date'])

data['Total'] = data['Imports'] + data['Exports']

# Retrieve latest date for sample country
dt = data.xs('Canada').index[-1]
ltdt = dtxt(dt)['mon1']
index = data.xs(dt, level=1).Total.sort_values(ascending=False).iloc[:101].index

final = data.loc[(index, dt),:].astype(int).droplevel('Date')
dt2 = dt - pd.DateOffset(months = 12)
data2 = data.loc[(index, dt2),:].astype(int).droplevel('Date')
data2.columns = [f'{k} ' for k in final.keys()]
final = final.join(data2).sort_values('Total', ascending=False)
final.index.name = ''
tbl = final.applymap('{:,.0f}'.format)
tbl.iloc[0, 0] = f'\${tbl.iloc[0, 0]}'
(tbl.iloc[:26].to_csv(data_dir / 'trade_partner.tex', sep='&', 
              lineterminator='\\\ ', quotechar=' '))

write_txt(text_dir / 'trade_date.txt', dtxt(dt)['mon1'])
write_txt(text_dir / 'trade_date2.txt', dtxt(dt2)['mon1'])

shval = (final['Total'].iloc[1:].sum() / 
         final['Total'].iloc[0]) * 100
tc = final.drop('Total, All Countries').index[:3]
tc_txt = f'{tc[0]}, {tc[1]}, and {tc[2]}'
t3sh = (final.loc[tc, 'Total'].sum() / 
        final.loc['Total, All Countries', 'Total'])
t3sh_txt = f'{t3sh*100:.1f} percent'

text = (f'In {ltdt}, trade with the top 25 trading partners (see '+
        f'table) comprises {shval:.1f} percent of total US trade in '+
        'goods. The top three US trading partners are '+
        f'{tc_txt}. These three countries account for {t3sh_txt} of '+
        f'US goods trade in {ltdt}. ')
write_txt(text_dir / 'trade_partner_sh.txt', text)
print(text)

In April 2024, trade with the top 25 trading partners (see table) comprises 99.5 percent of total US trade in goods. The top three US trading partners are Mexico, Canada, and China. These three countries account for 40.9 percent of US goods trade in April 2024. 


In [12]:
# Change text
ch = final['Total'].iloc[1:] - final['Total '].iloc[1:]

inc = (ch.iloc[:25] > 0).sum()
dec = (ch.iloc[:25] < 0).sum()
tch = ch.sort_values(ascending=False).index[0]
tch2 = ch.sort_values(ascending=False).index[1]
sch = ch.sort_values(ascending=False).index[-1]
cht = value_text(ch.loc[tch] / 1000, 'increase_by', 
                 ptype='billion', casual=True)
pch = (final.loc[tch, 'Total'] / final.loc[tch, 'Total '] - 1) * 100
pcht = value_text(pch, 'plain')

scht = value_text(ch.loc[sch] / 1000, 'increase_by', ptype='billion', 
                  casual=True).replace('fell', 'falling')
psc = (final.loc[sch, 'Total'] / final.loc[sch, 'Total '] - 1) * 100
psct = value_text(psc, 'plain').replace('negative ', 'a drop of ')

cht2 = value_text(ch.loc[tch2] / 1000, 'increase_of', 
                 ptype='billion')
pch2 = (final.loc[tch2, 'Total'] / final.loc[tch2, 'Total '] - 1) * 100
pcht2 = value_text(pch2, 'plain')

if dec > 0:
    txt = (f'The largest one-year decrease is with {sch}, with monthly trade '+
           f'{scht}, which is {psct}.')
else:
    txt = ('The second largest one-year rise in monthly trade was '+
           f'{cht2} with {tch2}, which is {pcht2}.')

only = 'all ' if inc == 25 else 'only ' if inc < 10 else ''

totc = final.loc['Total, All Countries']
totch = value_text((totc['Total'] / totc['Total '] - 1) * 100, casual=True)

text = (f'Over the year ending {ltdt}, nominal total trade '+
        f'increased among {only}{inc} of the top 25 trading partners. '+
        f'The largest one-year increase in total trade was with {tch}. '+
        f'Monthly trade with {tch} {cht}, or {pcht}. {txt} Total trade '+
        f'with all countries {totch} over the year.')
write_txt(text_dir / 'trade_partner_ch.txt', text)
print(text)

Over the year ending April 2024, nominal total trade increased among 21 of the top 25 trading partners. The largest one-year increase in total trade was with Mexico. Monthly trade with Mexico grew by 9.6 billion, or 15.2 percent. The largest one-year decrease is with China, with monthly trade falling by 2.9 billion, which is a drop of 6.4 percent. Total trade with all countries grew 7.4 percent over the year.


### Trade Levels, Latest in Detail

In [13]:
url = 'https://www.census.gov/econ_getzippedfile/?programCode=FTD'
r = requests.get(url)
zip_file = ZipFile(BytesIO(r.content))
with zip_file.open('FTD-mf.csv', 'r') as csvfile:
    tables = csvfile.read().split(b"\n\n\n")
    tables = [tbl.decode().replace('\r', '').split('\n') for tbl in tables]

In [14]:
d = {tbl[0]: pd.DataFrame([t.split(',') for t in tbl[1:]]).T.set_index(0).T 
     for tbl in tables}
cats = (d['CATEGORIES'].loc[:, ['cat_idx', 'cat_code']]
        .set_index('cat_idx').to_dict()['cat_code'])
tp = (d['TIME PERIODS'].loc[:, ['per_idx', 'per_name']]
        .set_index('per_idx').to_dict()['per_name'])
dtype = (d['DATA TYPES'].loc[:, ['dt_idx', 'dt_code']]
        .set_index('dt_idx').to_dict()['dt_code'])
data = pd.DataFrame()
data['CAT'] = d['DATA']['cat_idx'].dropna().map(cats)
data['DATE'] = d['DATA']['per_idx'].dropna().map(tp)
data['TYPE'] = d['DATA']['dt_idx'].dropna().map(dtype)
data['DATA'] = d['DATA']['val'].dropna().astype('int') / 1000 
data['SA'] = d['DATA']['is_adj']
data.index = pd.to_datetime(data.DATE)
df = pd.DataFrame()
for i in ['IMP', 'EXP', 'BAL']:
    df[i] = data.query('CAT == "BOPGS" & SA == "1" & TYPE == @i')['DATA']
df.loc['2015':].to_csv(data_dir / 'tradelt.csv', index_label='date')

In [ ]:
# result = pd.DataFrame()
# url0 = 'https://www.census.gov/econ/currentdata/export/csv'
# url1 = ('?programCode=FTD&timeSlotType=12&startYear=2015&'+
#         'endYear=2022&categoryCode=BOPGS&')
# for flow in ['EXP', 'IMP']:
#     url2 = (f'dataTypeCode={flow}&geoLevelCode=US&adjusted=yes&'+
#             'errorData=no&internal=false')
#     url = url0 + url1 + url2
#     s = pd.read_csv(url, skiprows=6).set_index('Period').dropna()
#     s.index = pd.to_datetime(s.index)
#     result[flow] = s['Value']

# balance = lambda x: x.EXP - x.IMP    
# df = result.loc['2015':].div(1000).assign(BAL = balance)
# df.to_csv(data_dir / 'tradelt.csv', index_label='date')

In [15]:
# Chart and text for latest monthly trade data
df = pd.read_csv(data_dir / 'tradelt.csv', index_col='date', 
                 parse_dates=True)

# End nodes for time series graph
res = df[['IMP', 'EXP']]
adj = node_adj(res)
#smax = res2.iloc[-1].idxmax()
#adj[smax] = adj[smax] + 0.35
cols = {'IMP': 'blue!80!violet', 'EXP': 'green!80!blue'}
date = {series: 'm' if series in ['IMP'] else None 
        for series in cols.keys()}
nodes  ='\n'.join([end_node(res[series], color, date=date[series], 
                            full_year=True, offset=adj[series]) 
                   for series, color in cols.items()])
write_txt(text_dir / 'tradelt_nodes.txt', nodes) 
node = end_node(df.BAL, 'red', date='m', offset=0.35, 
                full_year=True)
write_txt(text_dir / 'ballt.txt', node)

ltdt = dtxt(df.index[-1])['mon1']
ltmo = dtxt(df.index[-1])['mon3']
prdt = (dtxt(df.index[-2])['mon3'] 
        if df.index[-1].year == df.index[-2].year 
        else dtxt(df.index[-2])['mon1'])
dfs = [('lt', df.iloc[-1]), ('pr', df.iloc[-2]), 
       ('lt3', df.iloc[-3:].mean()), 
       ('pr3', df.iloc[-15:-12].mean()), 
       ('pr19', df.loc['2019'].mean())]
val = {name: srs.apply(lambda x: f'\${abs(x):.1f} billion') 
       for name, srs in dfs}
cols = [('IMP', 'blue!80!violet'), ('EXP', 'green!80!blue'),
        ('BAL', 'red')]
cline = {name: c_line(col) for name, col in cols}

# Context values
pop = (pd.read_csv(data_dir/'nipa20600.csv', index_col='date', 
                  parse_dates=True)['B230RC'])

emp = (pd.read_csv(data_dir / 'jobs_report_main2.csv', 
                   index_col='date', parse_dates=True)
        .loc['1989':, 'EMPsa'])

tpop = (df.multiply(1_000_000).divide(pop, axis=0).dropna()
          .applymap('\${:,.0f} per capita'.format))
temp = (df.multiply(1_000_000).divide(emp, axis=0).dropna()
          .applymap('\${:,.0f} per worker'.format))
imp_pc = tpop['IMP'].iloc[-1]
exp_pc = tpop['EXP'].iloc[-1]
exp_pw = temp['EXP'].iloc[-1]

text = (f'US goods and services imports total {val["lt"].IMP} in {ltdt}, '+
        f'following {val["pr"].IMP} in {prdt} {cline["IMP"]}. '+
        f'Imports average {val["lt3"].IMP} over the latest three months of data, '+
        f'and {val["pr3"].IMP} during the same months, one year prior. '+
        f'In 2019, monthly US imports average {val["pr19"].IMP}. For additional '+
        f'context, imports are equivalent to {imp_pc}, in '+
        'the latest month.\n\n'+
        f'The US exported {val["lt"].EXP} of goods and services in {ltdt}, '+
        f'following {val["pr"].EXP} in {prdt} {cline["EXP"]}. The three-month '+
        f'average was {val["lt3"].EXP} in {ltmo}, and {val["pr3"].EXP} '+
        f'one year prior. Exports were {val["pr19"].EXP} per month, '+
        'on average, in 2019. In the latest month, exports are equivalent '+
        f'to {exp_pc} or {exp_pw}. \n\n'+
        'Spending on imports exceeds payments '+
        f'received for exports, resulting in a trade deficit. In {ltmo}, '+
        f'the trade deficit was {val["lt"].BAL}, following {val["pr"].BAL} '+
        f'in {prdt} {cline["BAL"]}. Over the past three months, the average '+
        f'trade deficit is {val["lt3"].BAL}, compared to '+
        f'{val["pr3"].BAL} one year prior. '+
        f'In 2019, the average monthly trade deficit is {val["pr19"].BAL}.')
write_txt(text_dir / 'tradeltlevels.txt', text)
print(text)

US goods and services imports total \$338.2 billion in April 2024, following \$330.2 billion in March (see {\color{blue!80!violet}\textbf{---}}). Imports average \$334.5 billion over the latest three months of data, and \$321.5 billion during the same months, one year prior. In 2019, monthly US imports average \$258.8 billion. For additional context, imports are equivalent to \$1,005 per capita, in the latest month.

The US exported \$263.7 billion of goods and services in April 2024, following \$261.6 billion in March (see {\color{green!80!blue}\textbf{---}}). The three-month average was \$263.8 billion in April, and \$253.6 billion one year prior. Exports were \$212.2 billion per month, on average, in 2019. In the latest month, exports are equivalent to \$783 per capita or \$1,633 per worker. 

Spending on imports exceeds payments received for exports, resulting in a trade deficit. In April, the trade deficit was \$74.6 billion, following \$68.6 billion in March (see {\color{red}\tex

### Monthly trade data summary table

In [16]:
url = ('https://www.census.gov/foreign-trade/Press-Release/current_press_release/exh1.xlsx')
df = pd.read_excel(url, skipfooter=5, header=[4,5], index_col=0)
df = df.loc[~df.index.str.contains('-')]
y1 = df.index.name
y2 = df.index[12]
y3 = df.index[25]
i1 = df.index[:12] + ' 01, ' + y1
i2 = df.iloc[13:25].dropna().index.str.replace(' (R)', '', regex=False) + ' 01, ' + y2
i3 = df.iloc[26:].dropna().index.str.replace(' (R)', '', regex=False) + ' 01, ' + y3

df2 = df.dropna()
df2.index = pd.to_datetime([*i1, *i2, *i3])
df2.columns = [i[1] + ' ' + i[0] for i in df2.columns]
df2.columns = df2.columns.str.replace(' (1)', '', regex=False)
df2.to_csv(data_dir / 'trade_mo_summary.csv', index_label='date')

In [17]:
df2 = pd.read_csv(data_dir / 'trade_mo_summary.csv', 
                  index_col='date', parse_dates=True) / 1000
lt = df2.iloc[-3:].iloc[::-1].T
pryr = df2.iloc[-13].to_frame()
pryr2 = df2.iloc[-25].to_frame()
modt = pd.concat([lt, pryr, pryr2], axis=1)
cm = (df2.resample('QS').count() > 2)['Total Balance']
ltqtr = df2.resample('QS').mean().loc[cm[cm].index[-1]].to_frame()
prqtr = df2.resample('QS').mean().loc[cm[cm].index[-2]].to_frame()
pr2qtr = df2.resample('QS').mean().loc[cm[cm].index[-3]].to_frame()
qtrdt = pd.concat([ltqtr, prqtr, pr2qtr], axis=1)
modt.columns = [dtxt(i)['mon2'] for i in modt.columns]
qtrdt.columns = [dtxt(i)['qtr1'] for i in qtrdt.columns]
table = pd.concat([modt, qtrdt], axis=1)
table = table.applymap('{:.1f}'.format)
indent = [0, 2, 2, 1, 3, 3, 1, 3, 3]
refs = ['red', 0, 0, 'green!80!blue', 0, 0, 'blue!80!violet', 0, 0]
rt = [' \ ' + c_line(c, see=False) if c != 0 else '' for c in refs]
table.index = [f'\hspace{{{i}mm}} ' for i in indent] + table.index + rt
(table.to_csv(data_dir / 'trade_mo_summary.tex', sep='&', 
            lineterminator='\\\ ', quotechar=' '))

### World Bank Tariff Data (WDI)

In [ ]:
# from io import BytesIO
# from zipfile import ZipFile

# url = 'http://api.worldbank.org/v2/en/indicator/TM.TAX.MRCH.WM.AR.ZS?downloadformat=csv'

# r = requests.get(url)
# zip_file = ZipFile(BytesIO(r.content))
# files = zip_file.namelist()
# data_file = [file for file in files if file[0] != 'M']
# with zip_file.open(data_file[0], 'r') as csvfile:
#     df = pd.read_csv(csvfile, skiprows=4)
    
# data = df[df['Country Code'] == 'USA'].T.loc['1989':].dropna().iloc[:,0]
# data.index = pd.to_datetime([f'{year}-07-01' for year in data.index])
# data.name = 'tariff'
# data.index.name = 'date'
# data[pd.to_datetime('1994-07-01')] = np.nan
# data = data.sort_index()
# data.to_csv(data_dir / 'tariff.csv')
# write_txt(text_dir / 'tariff_node.txt', end_node(data, 'violet', percent=True))

# t89 = data.loc['1989-07-01']
# tlt = data.iloc[-1]
# ltyear = dtxt(data.index[-1])['year']

# text = ('World Bank staff \href{https://data.worldbank.org/indicator/TM.TAX.MRCH.WM.AR.ZS?locations=US&name_desc=true}{calculate} '+
#         'the weighted average applied tariff rate on US imports. The measure shows the trend '+
#         'in applied tariffs, factoring in the actual composition of imports by trading partner. '+
#         f'In 1989, the average tariff on US imports was {t89:.1f} percent. During the 1990s and 2000s, '+
#         'the US entered into various trade agreements, while China joined the WTO, obtaining '+
#         '\href{https://www.wto.org/english/thewto_e/whatis_e/tif_e/fact2_e.htm}{most-favored nation} '+
#         f'status with the US. In the latest data, covering {ltyear}, the average tariff was {tlt:.1f} '+
#         'percent.')
# write_txt(text_dir / 'tariff.txt', text)
# print(text)